In [20]:
!pip install kaggle
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [ ]:
import json
token = {"username":"fatemehy","key":"e6177841494493a99d3da8b61bab2fb1"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets list

- path is now set to: {/content}
ref                                                             title                                                size  lastUpdated          downloadCount  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge               COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-06-10 00:09:23          71785  
roche-data-science-coalition/uncover                            UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53          10483  
kwullum/fatal-police-shootings-in-the-us                        Fatal Police Shootings in the US                      1MB  2017-09-22 19:18:21          12617  
Cornell-University/arxiv                                        ArXiv Metadata                                        2GB  2020-05-06 23:18:35              2  
sudalai

In [ ]:
!kaggle datasets list -s medical-appointment
!kaggle datasets download -d joniarroba/noshowappointments       -p /content
!unzip \*.zip

ref                                                      title                                        size  lastUpdated          downloadCount  
-------------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  
joniarroba/noshowappointments                            Medical Appointment No Shows                  2MB  2017-08-20 23:49:38          25607  
afflores/medical-appointment                             Medical Appointment                         538KB  2017-12-17 17:27:08            302  
frederickferguson/hair-salon-no-show-data-set            Hair Salon No-Show Dataset                   61KB  2018-08-01 15:24:06           1243  
kuroski/medicalappointmentnoshown                        medical-appointment-no-shown                  2MB  2019-04-05 20:24:52             49  
vinodkumarcvk/healthcareticketingsystem                  healthcareticketingsystem                     7MB  2017-12-02 09:21:55   

In [21]:
# Load data into Pandas Dataframe
import pandas as pd
medical_appointment = pd.read_csv('KaggleV2-May-2016.csv')
medical_appointment

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [22]:
#Does the dataset include any missing values? If so, delete the missing values entries
medical_appointment.isnull().sum(axis = 0)

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [23]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(medical_appointment['No-show'])
medical_appointment['No-show']= encoder.transform(medical_appointment['No-show'])

medical_appointment.dropna()



,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,0
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,0
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,0
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,0


In [24]:
#target_value
no_show = medical_appointment["No-show"].value_counts()
print(no_show)
print(medical_appointment["No-show"])
#record target value
# Percent_no_show = no_show["Yes"]/ no_show.sum() * 100
# print("Percent who didn't show up to their appointment:",Percent_no_show )
medical_appointment['No-show'].replace("No", 0,inplace=True)
medical_appointment['No-show'].replace("Yes", 1,inplace=True)
# medical_appointment['Gender'].replace("F", 1,inplace=True)
# medical_appointment['Gender'].replace("M", 0,inplace=True)
# from sklearn.preprocessing import OneHotEncoder
# encoded_X = OneHotEncoder().fit_transform(selected_X).toarray()
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(medical_appointment['Gender'])
medical_appointment['Gender']= encoder.transform(medical_appointment['Gender'])

0    88208
1    22319
Name: No-show, dtype: int64
0         0
1         0
2         0
3         0
4         0
         ..
110522    0
110523    0
110524    0
110525    0
110526    0
Name: No-show, Length: 110527, dtype: int64


In [25]:

medical_appointment2=medical_appointment[['No-show','Gender','Age','Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']]

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(medical_appointment2.iloc[:,1:])
y = medical_appointment2.iloc[:, 0].values
medical_appointment2.head()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=1) 

# print(y_test,y_pred)



In [ ]:
#trying Decision tree classifier with criterion='entropy'
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='gini', random_state=0).fit(X_train, y_train)
score = model.score(X_test, y_test)
print('accuracy score: ',score)
y_pred = model.predict(X_test)
y_pred = model.predict(X_test)
from sklearn.metrics import recall_score
print('recall score: '+ str(recall_score(y_test, y_pred)))
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=10, random_state=100)
results_kfold = cross_val_score(model, X, y, cv=kfold)
print("Accuracy of validation: %.2f%%" % (results_kfold.mean()*100.0)) 
# 




#trying Decision tree classifier with criterion='entropy'

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(X_train, y_train)
score = model.score(X_test, y_test)
print('accuracy score: ',score)
y_pred = model.predict(X_test)
y_pred = model.predict(X_test)
from sklearn.metrics import recall_score
print('recall score: '+ str(recall_score(y_test, y_pred)))
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=10, random_state=100)
results_kfold = cross_val_score(model, X, y, cv=kfold)
print("Accuracy of validation: %.2f%%" % (results_kfold.mean()*100.0)) 




accuracy score:  0.8002714318027595
recall score: 0.027225247249694412
[[35135    76]
 [ 8754   245]]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy of validation: 79.51%
accuracy score:  0.8002714318027595
recall score: 0.027225247249694412
[[35135    76]
 [ 8754   245]]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy of validation: 79.51%


In [34]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50, criterion='entropy', random_state=0).fit(X_train, y_train)
score = model.score(X_test, y_test)
print('accuracy score: ',score)
y_pred = model.predict(X_test)
from sklearn.metrics import recall_score
print('recall score: '+ str(recall_score(y_test, y_pred, average='weighted')))
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=10, random_state=100)
results_kfold = cross_val_score(model, X, y, cv=kfold)
print("Accuracy of validation: %.2f%%" % (results_kfold.mean()*100.0)) 

# search for an optimal value of K for KNN
# empty list to store scores
k_scores = []

# 1. we will loop through reasonable values of k
for k in range(10,100,20):
    # 2. run KNeighborsClassifier with k neighbours
    RFC = RandomForestClassifier(n_estimators=k)
    # 3. obtain cross_val_score for KNeighborsClassifier with k neighbours
    scores = cross_val_score(RFC, X, y, cv=10, scoring='accuracy')
    # 4. append mean of scores for k neighbors to k_scores list
    k_scores.append(scores.mean())


print(k_scores)
print('best k:',10+(np.asarray(k_scores,dtype=float)).argmax()*20)



accuracy score:  0.8003166704365529
recall score: 0.8003166704365529
[[35076   135]
 [ 8693   306]]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy of validation: 79.45%
[0.7938512737578447, 0.7942765148698376, 0.7944574718612218, 0.7944212735865908, 0.7944936537635809]
best k: 90
